# Part 1

Implement a real-time Stock trading engine for matching Stock Buys with Stock Sells.<br>
1. Write an ‘addOrder’ function that will have the following parameters:<br>
      ‘Order Type’ (Buy or Sell), ‘Ticker Symbol’, ‘Quantity’, ‘Price’<br>
      Support 1,024 tickers (stocks) being traded.<br>
      Write a wrapper to have this ‘addOrder’ function randomly execute with different parameter values to simulate active stock transactions.<br>

Program flow:<br><br>
1. Call the addOrder function to simulate xx number of records<br>
2. In the matchOrder:<br>
    a. Create empty lists for manipulating the stock buys and sells<br>
    b. Capture a snapshot of the all records from point 1<br>
    c. Create a list of the lowest sell stock<br>
    d. Find out the if the buy price is greater than the lowest sell, then only initiate a buy<br>

In [1]:
# Step 1: Load necessary libraries

# sleep for certain duration
import time
# to capture the datetime of order
import datetime
# Randomize the symbols, quantities, price
import random
import threading

In [2]:
# Step 2 : Define any variables and function for error logging

# List of 1,024 ticker symbols for simulation
ticker_symbols = ["Symbol-" + str(i) for i in range(1, 1025)]

# Use a list to store orders and process in a FIFO(first in first out)
order_book = []

In [3]:
# function to write any errors to log file
def log_error(error_message):
    """
    Function to write error to log file
    """
    with open("error_log.txt", "a") as f:
        # format with  current date and time along side with error message
        f.write(f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {error_message}\n")

In [4]:
# Step 3 : addOrder function

# function addOrder with parameters
def addOrder(order_type, symbol, quantity, price):
    """
    Function to add the simulated orders to the order_store
    :param order_type: Buy or Sell
    :param symbol: Symbol of ticker
    :param quantity: number of shares
    :param price: Price per share
    """
    try:
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  
        order = (order_type, symbol, quantity, price, timestamp)
        order_book.append(order)        
        print("Order Added : ", order)
    except Exception as e:
        log_error(f"Error in function addOrder : {str(e)}")
    

In [5]:
# Step 4 : Simulate active stock transaction function

def simulate_activestock_transactions(number_of_transactions, delay_time):
    """
    Function to simulate active stock transactions with random orders.
    :param num_transactions: Number of transactions to simulate
    :param delay: Time delay between transactions
    """
    try:
        for _ in range(number_of_transactions):
            #randomly selects between buy(0) and sell(1)
            order_type = "Buy" if random.randint(0,1) == 0 else 'Sell'
            # Randomly selects symbols till 1024
            symbol_selected = ticker_symbols[random.randint(0, 1023)]
            # Randomly selects qty between 1 and 500, using randint since we
            # are dealing with whole numbers
            quantity = random.randint(1,500)
            # Randomly selects prices between $10 and $300, rounding to 2 decimals,
            # and uniform generates a random floating point number
            price = round(random.uniform(10, 300), 2)

            # call the function to add stock
            addOrder(order_type, symbol_selected, quantity, price)

            # Delay in transaction, in seconds
            time.sleep(delay_time)
    except Exception as e:
        log_error(f"Error in function simulate_activestock_transactions : {str(e)}")

In [6]:
# Step 5 : Call the function to simulate transactions
    
# Call the function
# Add test orders to ensure matches
#addOrder("Buy", "Symbol-500", 100, 50.00)  # Buy order at $50
#addOrder("Sell", "Symbol-500", 100, 45.00)  # Sell order at $45 (should match)
#addOrder("Buy", "Symbol-300", 200, 30.00)  # Buy order at $30
#addOrder("Sell", "Symbol-300", 150, 28.00)  # Sell order at $28 (should match)
addOrder("Buy", "Symbol-400", 200, 45.00)  # Sell order with no match
addOrder("Sell", "Symbol-400", 200, 50.00)  # No action since price is less than sell
addOrder("Sell", "Symbol-401", 200, 50.00)  # No action since price is less than sell

# Simulate additional random stock transactions
try:
    simulate_activestock_transactions(10, 0.5)
except Exception as e:
    log_error(f"Error in calling function simulate_activestock_transactions : {str(e)}")

Order Added :  ('Buy', 'Symbol-400', 200, 45.0, '2025-03-07 12:40:23')
Order Added :  ('Sell', 'Symbol-400', 200, 50.0, '2025-03-07 12:40:23')
Order Added :  ('Sell', 'Symbol-401', 200, 50.0, '2025-03-07 12:40:23')
Order Added :  ('Buy', 'Symbol-360', 117, 255.29, '2025-03-07 12:40:23')
Order Added :  ('Buy', 'Symbol-540', 102, 34.49, '2025-03-07 12:40:23')
Order Added :  ('Sell', 'Symbol-352', 34, 204.07, '2025-03-07 12:40:24')
Order Added :  ('Sell', 'Symbol-663', 32, 107.0, '2025-03-07 12:40:24')
Order Added :  ('Buy', 'Symbol-45', 170, 298.43, '2025-03-07 12:40:25')
Order Added :  ('Buy', 'Symbol-179', 340, 205.63, '2025-03-07 12:40:25')
Order Added :  ('Buy', 'Symbol-67', 315, 147.61, '2025-03-07 12:40:26')
Order Added :  ('Buy', 'Symbol-526', 61, 104.42, '2025-03-07 12:40:26')
Order Added :  ('Sell', 'Symbol-402', 21, 201.35, '2025-03-07 12:40:27')
Order Added :  ('Sell', 'Symbol-138', 457, 19.11, '2025-03-07 12:40:27')


# Part 2

2. Write a ‘matchOrder’ function, that will match Buy & Sell orders with the following criteria:<br>
      Buy price for a particular ticker is greater than or equal to lowest Sell price available then.<br>
      Write your code to handle race conditions when multiple threads modify the Stock order book, as run in real-life, by multiple stockbrokers. <br>
     

In [7]:
def matchOrder():
    """
    Buy price must be >= the lowest available Sell price.
    Trades execute at the lowest sell price.
    """
    try:
        matched_orders = []
        buy_orders = []
        sell_orders = []
        matched_sell_orders = set()
        matched_buy_orders = set()
        
        global order_book
        processing_orders = order_book[:]
        order_book.clear()
        
        for order in processing_orders:
            if order[0] == "Buy":
                buy_orders.append(order)
            elif order[0] == "Sell":
                sell_orders.append(order)
        
        filtered_sell_orders = []
        for sell_order in sell_orders:
            for idx in range(len(filtered_sell_orders)):
                if filtered_sell_orders[idx][1] == sell_order[1]:
                    if sell_order[3] < filtered_sell_orders[idx][3]:
                        filtered_sell_orders[idx] = sell_order
                    break
            else:
                filtered_sell_orders.append(sell_order)
        
        for buy_order in buy_orders[:]:  # Iterate over a copy to allow modification
            for min_sell in filtered_sell_orders[:]:  # Ensure buy price meets condition
                if min_sell[1] == buy_order[1] and buy_order[3] >= min_sell[3]:  # Buy price must be >= lowest sell price
                    execution_price = min_sell[3]
                    matched_orders.append(("Buy Order", *buy_order, execution_price))
                    matched_orders.append(("Sell Order", *min_sell))
                    matched_sell_orders.add(tuple(min_sell))
                    matched_buy_orders.add(tuple(buy_order))
                    filtered_sell_orders.remove(min_sell)
                    sell_orders.remove(min_sell)  # Remove matched sell order
                    buy_orders.remove(buy_order)  # Remove matched buy order
                    print(f"Matched: {buy_order[1]} | Price: {execution_price}")
                    break
        
        print("\nCompleted Transactions:")
        for match in matched_orders:
            print(match)
        
        with open("matched_orders.txt", "w") as f:
            for match in matched_orders:
                f.write(str(match) + "\n")
        
        print("\nOrders Remaining in Order Book:")
        unmatched_orders = [order for order in buy_orders + sell_orders if tuple(order) not in matched_orders]
        order_book = [order for order in unmatched_orders if order[0] == 'Buy' and tuple(order) not in matched_orders]
        
        for order in order_book:
            print(order)
        
    except Exception as e:
        log_error(f"Error in matchOrder: {str(e)}")

In [8]:
def process_orders():
    """
    Continuously process stock orders without explicit locks.
    """
    while True:
        try:
            # Process only if there are orders
            if order_book:  
                # No lock used, ensuring concurrent order processing
                matchOrder()  
        except Exception as e:
            log_error(f"Error in function process_orders: {str(e)}")
        # Small delay to allow order accumulation
        time.sleep(1)

In [9]:
# This creates a separate background thread that continuously runs the process_orders() function, 
# ensuring that stock orders are matched in real-time without blocking the main program. 
# The daemon=True parameter makes the thread run in the background and automatically 
# stop when the main program exits.

order_matching_thread = threading.Thread(target=process_orders, daemon=True)
order_matching_thread.start()


Completed Transactions:

Orders Remaining in Order Book:
('Buy', 'Symbol-400', 200, 45.0, '2025-03-07 12:40:23')
('Buy', 'Symbol-360', 117, 255.29, '2025-03-07 12:40:23')
('Buy', 'Symbol-540', 102, 34.49, '2025-03-07 12:40:23')
('Buy', 'Symbol-45', 170, 298.43, '2025-03-07 12:40:25')
('Buy', 'Symbol-179', 340, 205.63, '2025-03-07 12:40:25')
('Buy', 'Symbol-67', 315, 147.61, '2025-03-07 12:40:26')
('Buy', 'Symbol-526', 61, 104.42, '2025-03-07 12:40:26')

Completed Transactions:

Orders Remaining in Order Book:
('Buy', 'Symbol-400', 200, 45.0, '2025-03-07 12:40:23')
('Buy', 'Symbol-360', 117, 255.29, '2025-03-07 12:40:23')
('Buy', 'Symbol-540', 102, 34.49, '2025-03-07 12:40:23')
('Buy', 'Symbol-45', 170, 298.43, '2025-03-07 12:40:25')
('Buy', 'Symbol-179', 340, 205.63, '2025-03-07 12:40:25')
('Buy', 'Symbol-67', 315, 147.61, '2025-03-07 12:40:26')
('Buy', 'Symbol-526', 61, 104.42, '2025-03-07 12:40:26')

Completed Transactions:

Orders Remaining in Order Book:
('Buy', 'Symbol-400', 200,

 Also, use lock-free data structures.
 Do not use any dictionaries, maps or equivalent data structures. Essentially there should be no ‘import’-s nor ‘include’-s nor similar construct relevant to the programming language you are using that provides you dictionary, map or equivalent data structure capability. In essence, you are writing the entire code. Standard language-specific non data structure related items are ok, but try to avoid as best as you can.
      Write your ‘matchOrder’ function with a time-complexity of O(n), where 'n' is the number of orders in the Stock order book.